In [66]:
%matplotlib inline
import nba_py as nba
from nba_py import player
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
import math

We will look into some general metrics and see how the overall trend compares for shot clock time, playoffs, 4th quarter and determine which metrics are important for determining clutchness

In [ ]:
#loop through and get shot percentages for players between regular season and 4th quarter

#Also need to append player ID/Names

In [258]:
def get_player_data(pl_id):
    #Takes in player id and returns dataframe with player's historical data under specific conditions
    #We are observing shot percentages, rebounds, assists, turnovers and blocks
    #overall year by year data
    p_df1 = player.PlayerYearOverYearSplits(pl_id, period = '0').by_year()
    p_df1 = p_df1[['GROUP_VALUE','FG_PCT','FG3_PCT', 'FT_PCT', 'OREB', 'DREB',
                           'REB', 'AST', 'TOV', 'STL', 'BLK']]
    if p_df1.empty == False:
        p_df1 = p_df1.groupby(['GROUP_VALUE']).mean()
        p_df1 = p_df1.reset_index()
    
    p_df2 = player.PlayerYearOverYearSplits(pl_id, period = '4').by_year()
    p_df2 = p_df2[['GROUP_VALUE','FG_PCT','FG3_PCT', 'FT_PCT', 'OREB', 'DREB',
                           'REB', 'AST', 'TOV', 'STL', 'BLK']]
    if p_df2.empty == False:
        p_df2 = p_df2.groupby(['GROUP_VALUE']).mean()
        p_df2 = p_df2.reset_index()
    p_df2.columns = ['GROUP_VALUE','4Q_FG_PCT','4Q_FG3_PCT', '4Q_FT_PCT', '4Q_OREB', '4Q_DREB',
                        '4Q_REB', '4Q_AST', '4Q_TOV', '4Q_STL', '4Q_BLK']

    p_main = pd.merge(p_df1,p_df2, how = 'left', on ='GROUP_VALUE')

    p_df3 = player.PlayerYearOverYearSplits(pl_id, season_type = 'Playoffs').by_year()
    p_df3 = p_df3[['GROUP_VALUE','FG_PCT','FG3_PCT', 'FT_PCT', 'OREB', 'DREB',
                        'REB', 'AST', 'TOV', 'STL', 'BLK']]
    if p_df3.empty == False:
        p_df3 = p_df3.groupby(['GROUP_VALUE']).mean()
        p_df3 = p_df3.reset_index()
    p_df3.columns = ['GROUP_VALUE','PO_FG_PCT','PO_FG3_PCT', 'PO_FT_PCT', 'PO_OREB', 'PO_DREB',
                        'PO_REB', 'PO_AST', 'PO_TOV', 'PO_STL', 'PO_BLK']

    p_main = pd.merge(p_main, p_df3, how='left', on = 'GROUP_VALUE')
    
    p_df4 = player.PlayerYearOverYearSplits(pl_id, shot_clock_range = '4-0 Very Late').by_year()
    p_df4 = p_df4[['GROUP_VALUE','FG_PCT','FG3_PCT', 'FT_PCT', 'OREB', 'DREB',
                        'REB', 'AST', 'TOV', 'STL', 'BLK']]
    if p_df4.empty == False:
        p_df4 = p_df4.groupby(['GROUP_VALUE']).mean()
        p_df4 = p_df4.reset_index()
    p_df4.columns = ['GROUP_VALUE','SC_FG_PCT','SC_FG3_PCT', 'SC_FT_PCT', 'SC_OREB', 'SC_DREB',
                        'SC_REB', 'SC_AST', 'SC_TOV', 'SC_STL', 'SC_BLK']

    p_main = pd.merge(p_main, p_df4, how='left', on = 'GROUP_VALUE')
    p_main['PERSON_ID'] = [pl_id]*len(p_main)
    
    return (p_main)


In [252]:
plist = player.PlayerList(league_id='00', season='2016-17')
player_ids = plist.info()['PERSON_ID']

In [273]:
main_df = pd.DataFrame()
for cur_player in tqdm.tqdm(player_ids.values):
    cur_df = get_player_data(cur_player)
    main_df = main_df.append(cur_df)


100%|██████████| 486/486 [1:11:58<00:00,  8.63s/it]


In [275]:
df = pd.merge(main_df,plist.info()[['PERSON_ID','DISPLAY_FIRST_LAST']], how='left', on='PERSON_ID')

In [277]:
df.to_csv('overall_df.csv')

In [278]:
df.head()

,GROUP_VALUE,FG_PCT,FG3_PCT,FT_PCT,OREB,DREB,REB,AST,TOV,STL,...,SC_FT_PCT,SC_OREB,SC_DREB,SC_REB,SC_AST,SC_TOV,SC_STL,SC_BLK,PERSON_ID,DISPLAY_FIRST_LAST
0,2016-17,0.393000,0.381,0.898000,0.3,1.000000,1.300000,0.600000,0.500000,0.500000,...,0,0,0,0,0,0,0,0,203518,Alex Abrines
1,2012-13,0.560000,0.500,0.816000,1.0,1.600000,2.700000,0.400000,0.600000,0.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203112,Quincy Acy
2,2013-14,0.456333,0.289,0.650667,1.0,2.033333,3.033333,0.466667,0.433333,0.433333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203112,Quincy Acy
3,2014-15,0.459000,0.300,0.784000,1.2,3.300000,4.400000,1.000000,0.900000,0.400000,...,0,0,0.1,0.1,0.1,0,0,0,203112,Quincy Acy
4,2015-16,0.556000,0.388,0.735000,1.1,2.100000,3.200000,0.500000,0.500000,0.500000,...,0,0,0.1,0.1,0,0.1,0.1,0,203112,Quincy Acy


In [301]:
print('Regular field goal percentage mean: ' + str(df[df['GROUP_VALUE'] == '2016-17']['FG_PCT'].mean()))
print('Shot clock low field goal percentage mean: ' + str(df[df['GROUP_VALUE'] == '2016-17']['SC_FG_PCT'].mean()))
      

Regular Field Goal Percentage mean: 0.43922959533607653
Shot Clock Low Field Goal Percentage mean: 0.33002233677


In [302]:
print('Regular 3 point percentage mean: ' + str(df[df['GROUP_VALUE'] == '2016-17']['FG3_PCT'].mean()))
print('Shot clock low field goal percentage mean: ' + str(df[df['GROUP_VALUE'] == '2016-17']['SC_FG3_PCT'].mean()))

Regular 3 point percentage mean: 0.276814986282579
Shot clock low field goal percentage mean: 0.212512199313


In [303]:
print('Regular free throw percentage mean: ' + str(df[df['GROUP_VALUE'] == '2016-17']['FT_PCT'].mean()))
print('Playoff free throw percentage mean: ' + str(df[df['GROUP_VALUE'] == '2016-17']['PO_FT_PCT'].mean()))
print('4th Quarter free throw percentage mean: ' + str(df[df['GROUP_VALUE'] == '2016-17']['4Q_FT_PCT'].mean()))

Regular free throw percentage mean: 0.716347908093278
Playoff free throw percentage mean: 0.625744186047
4th Quarter free throw percentage mean: 0.6919106652949248


In [314]:
df[df['GROUP_VALUE'] == '2016-17'].columns

Index(['GROUP_VALUE', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'OREB', 'DREB', 'REB',
       'AST', 'TOV', 'STL', 'BLK', '4Q_FG_PCT', '4Q_FG3_PCT', '4Q_FT_PCT',
       '4Q_OREB', '4Q_DREB', '4Q_REB', '4Q_AST', '4Q_TOV', '4Q_STL', '4Q_BLK',
       'PO_FG_PCT', 'PO_FG3_PCT', 'PO_FT_PCT', 'PO_OREB', 'PO_DREB', 'PO_REB',
       'PO_AST', 'PO_TOV', 'PO_STL', 'PO_BLK', 'SC_FG_PCT', 'SC_FG3_PCT',
       'SC_FT_PCT', 'SC_OREB', 'SC_DREB', 'SC_REB', 'SC_AST', 'SC_TOV',
       'SC_STL', 'SC_BLK', 'PERSON_ID', 'DISPLAY_FIRST_LAST'],
      dtype='object')

In [315]:
df[df['GROUP_VALUE'] == '2016-17']['BLK'].mean()

0.3913923182441697

In [316]:
df[df['GROUP_VALUE'] == '2016-17']['PO_BLK'].mean()

0.38093023255813935

We clearly have a discrepency in statistics under specific conditions. Are these Significant!? RUN TEST. Can we determine which types of players are prone to make these bad judgements in crunch time? (Types of shots). How do we quantify clutchness and what other metrics will support the argument that one player is more clutch than another? (Look at average drop in performance compared to a specific player) 